In [62]:
!pip install -q tabula-py
!pip install PyMuPDF
!pip install selenium requests beautifulsoup4

In [63]:
import os
import re
import tabula
import pandas as pd
import numpy as np
import fitz  # module from PyMuPDF

In [64]:
# Set the JAVA_HOME environment variable to the Java installation directory
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk/libexec/openjdk.jdk"

In [65]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [66]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import os
import re
import time

# Function to convert month string to mm format
def month_to_mm(month_str):
    return {
        'January': '01', 'February': '02', 'March': '03', 'April': '04',
        'May': '05', 'June': '06', 'July': '07', 'August': '08',
        'September': '09', 'October': '10', 'November': '11', 'December': '12'
    }.get(month_str, '')

options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run headless Chrome (no GUI)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920x1080')

driver = webdriver.Chrome(options=options)

try:
    driver.get("https://company.meralco.com.ph/news-and-advisories/rates-archives")

    # Ask user for month and year
    #month = input("Enter the month (e.g., January): ").strip().title()
    #year = input("Enter the year (e.g., 2023): ").strip()
    month = "January"
    year = "2023"
    month_mm = month_to_mm(month)

    # Click "Show more" until the desired date is found or no more items are available
    date_found = False
    while not date_found:
        try:
            show_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//a[@class='btn btn-bordered load-more' and @title='Go to next page']"))
            )
            show_more_button.click()
            time.sleep(2)  # Wait for content to load

            rows = driver.find_elements(By.XPATH, "//tr[.//span[contains(text(), 'Summary Schedule of Rates') or contains(text(), 'Summary of Schedule of Rates')]]")

            for row in rows:
                try:
                    # Find the date in the same row
                    date_element = row.find_element(By.XPATH, ".//td[contains(@class, 'views-field-field-date-created')]")
                    date_text = date_element.text.strip()

                    # Convert date text to "mm-yy" format
                    match = re.search(r'(\w+)\s+(\d{4})', date_text)
                    if match:
                        month_str, year_str = match.groups()
                        if month_str == month and year_str == year:
                            # Find the PDF link
                            pdf_link_element = row.find_element(By.XPATH, ".//a[contains(@class, 'btn-bordered-orange') and contains(@href, '.pdf')]")
                            pdf_link = pdf_link_element.get_attribute('href')
                            date_found = True  # Set flag to true if date is found
                            break

                    else:
                        print(f"Date format not recognized for text: {date_text}")
                        continue  # Skip this row if date format is not recognized

                except Exception as e:
                    print(f"Error processing row: {e}")

        except Exception as e:
            print("No more items available or error clicking 'Show more'")
            break  # Break the loop if no more "Show more" button is found or error occurs

    # Download the PDF if a link was found

    folder_path = 'Use Case'

    os.makedirs(folder_path, exist_ok=True)

    if pdf_link:
        response = requests.get(pdf_link)
        
        if response.status_code == 200:
            # Create the filename with the dynamic month and year
            pdf_name = f"Summary_Schedule_of_Rates_{month_mm}-{year[-2:]}.pdf"
            
            # Create the full file path
            file_path = os.path.join(folder_path, pdf_name)
            
            # Save the PDF to the specified path
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f"Successfully downloaded PDF from {pdf_link} and saved to {file_path}")
        else:
            print(f"Failed to download PDF from {pdf_link}, status code: {response.status_code}")
    else:
        print(f"No PDF found for {month} {year}")

finally:
    driver.quit()

Successfully downloaded PDF from https://meralcomain.s3.ap-southeast-1.amazonaws.com/2023-01/01-2023_rate_schedule.pdf and saved to Use Case/Summary_Schedule_of_Rates_01-23.pdf


In [67]:
headers_and_subheaders = {
    "Customer Subclass": "",
    "Generation Charge": "per kWh",
    "Prev. Mos' Adj on Gen": "per kWh",
    "Power Act Reduction": "per kWh",
    "Transmission Charge": "per kWh, per kW",
    "System Loss Charge": "per kWh",
    "Distribution Charge": "per kWh, per kW",
    "Supply Charge": "per kWh, per cust/mo",
    "Metering Charge": "per kWh, per cust/mo",
    "Reset Cost Adjustment": "per kWh",
    "Distribution Rate True-Up": "per kWh",
    "Dist Rate True-Up 1": "per kWh",
    "Dist Rate True-Up 2": "per kWh",
    "Dist Rate True-Up 3": "per kWh",
    "Dist Rate True-Up 4": "per kWh",
    "For Non-lifeline: Lifeline Subsidy": "per kWh",
    "For Lifeline: Applicable Discounts": "%",
    "Lifeline Rate Subsidy": "per kWh",
    "Senior Citizen Subsidy": "per kWh",
    "Current RPT Charge": "per kWh",
    "UC-ME":"per kWh",
    "UC-ME for (NPC-SPUG)": "per kWh",
    "UC-ME for (RED-CI)": "per kWh",
    "UC-EC": "per kWh",
    "UC-SCC": "per kWh",
    "UC-SD": "per kWh",
    "Fit-All (Renewable)": "per kWh",
    "Cross Subsidy Charge": "per kWh",
    "Current RPT": "per kWh",
    "Lifeline Discount": "%",
    "Special Discount": "%",
    "Power Factor Adj": "Penalty, Disc",
    "Prev. Mos' Adj on Gen Cost": "per kWh"
}

In [70]:
date_part = pdf_name.split('_')[-1].replace('.pdf', '')

file_path = os.path.join(folder_path, pdf_name)

df = tabula.read_pdf(file_path, lattice=True, pages=1)[0]

In [71]:
def clean_column_name(col):
  col = col.replace('\r', ' ')  # Replace '\r' with a space
  col = re.sub(r'\s+', ' ', col)  # Replace multiple spaces with a single space
  # Special handling for "Dist Rate True-Up"
  if any(rate in col for rate in ['Dist Rate True-Up', 'Distribution Rate True-Up']):
      # Replace only '\r' and extra spaces, keep digits
      col = re.sub(r'[^\w\s\d-]', '', col)

  else:
      col = re.sub(r'\s*\d+\s*', ' ', col) # Remove superscripts

  return col.strip()  # Strip leading/trailing whitespace

In [72]:
def count_true_up_occurrences(columns):
    true_up_counter = 0
    for col in columns:
        if 'Up' in col:
            true_up_counter += 1
    return true_up_counter

In [73]:
def convert_column_names(columns):
    true_up_count = count_true_up_occurrences(columns)
    renamed_columns = []

    for col in columns:
        if 'Up' in col:
            if true_up_count == 1:
                renamed_columns.append("Distribution Rate True-Up")
            else:
                match = re.search(r'True-\s*Up\s*(\d)', col)
                if match:
                    num = match.group(1)
                    renamed_columns.append(f"Dist Rate True-Up {num}")
        else:
            renamed_columns.append(col)

    return renamed_columns

In [74]:
def process_headers(df):
  df.columns = convert_column_names(df.columns)
  df.columns = [clean_column_name(col) for col in df.columns]

  # Ensure unique column names by appending the DataFrame index if needed
  df.columns = [f"{col}_{i}" if col in df.columns[:i] else col for i, col in enumerate(df.columns)]

  # Clean the first row in a similar way and assign it back to the DataFrame
  df.iloc[0] = [clean_column_name(str(value)) for value in df.iloc[0]]

  non_null_values = df.iloc[0].dropna().values
  non_null_values = [val for val in non_null_values if val != 'nan']

  if "Universal Charge" in df.columns:
      universal_charge_index = df.columns.get_loc("Universal Charge")

      # Replace "Universal Charge" with the first non-null value
      df.columns.values[universal_charge_index] = non_null_values[0]

      # Insert any additional non-null values into the column headers
      remaining_values = non_null_values[1:]

      if remaining_values:
          # Split the columns into parts: before, at, and after "Universal Charge"
          before = df.columns[:universal_charge_index + 1]
          after = df.columns[universal_charge_index + 1:]

          # Create new columns with the remaining values inserted after the "Universal Charge" replacement
          new_columns = list(before) + remaining_values + list(after[:-len(remaining_values)])
          df.columns = new_columns

  # Replace "Lifeline Eligibility" with "For Non-lifeline: Lifeline Subsidy" and insert "For Lifeline: Applicable Discounts"
  if "Lifeline Eligibility" in df.columns:
      lifeline_index = df.columns.get_loc("Lifeline Eligibility")
      df.columns.values[lifeline_index] = "For Non-lifeline: Lifeline Subsidy"

      df.columns = list(df.columns[:lifeline_index + 1]) + ["For Lifeline: Applicable Discounts"] + list(df.columns[lifeline_index + 1:-1])

  # Replace column containing "SUMMARY" with "Customer Subclass"
  summary_column = next((col for col in df.columns if "SUMMARY" in col), None)
  if summary_column:
      df.rename(columns={summary_column: "Customer Subclass"}, inplace=True)

  # Prepare lists for multiindex tuples
  multi_index_headers = []

  for col in df.columns:
    if col in headers_and_subheaders:
        subheader = headers_and_subheaders[col].split(", ")
        for sub in subheader:
            multi_index_headers.append((col, sub))

  multi_index = pd.MultiIndex.from_tuples(multi_index_headers, names=['Charge', 'Unit'])

  df.columns = multi_index

  return df

In [75]:
df = process_headers(df)

In [76]:
def crop_dataframe(df):
  start_index = df[df.iloc[:, 0] == 'Residential'].index[0]
  end_index = df[df.iloc[:, 0] == '400 W HPS (or equivalent)'].index[0]

  df = df.iloc[start_index+1:end_index + 1]
  df.reset_index(drop=True, inplace=True)

  return df

In [77]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

cleaned_dataframes = []

# Convert date_str to a datetime object
date_obj = datetime.strptime(date_part, '%m-%y')

# Calculate Supply Period Start and End dates
supply_period_start = date_obj - relativedelta(months=1)
supply_period_start = supply_period_start.replace(day=26)
supply_period_start_str = supply_period_start.strftime('%m-%d-%y')

supply_period_end = date_obj.replace(day=25)
supply_period_end_str = supply_period_end.strftime('%m-%d-%y')

# Add the columns
df['Supply Period Start'] = supply_period_start_str
df['Supply Period End'] = supply_period_end_str
df['Supply Period'] = date_part

df = crop_dataframe(df)

In [78]:
df['Date'] = date_part

In [79]:
def is_valid_float(value):
  try:
      return not pd.isna(value) and float(value) is not None
  except ValueError:
      return False

is_valid_float_df = df.applymap(is_valid_float)

# Keep rows where there is at least one valid float-representing string
df = df[is_valid_float_df.any(axis=1)]

/var/folders/5w/f5z12zyx4tn68kmx7wk0wvv40000gn/T/ipykernel_9615/3132154341.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  is_valid_float_df = df.applymap(is_valid_float)


In [80]:
def clean_subclass(subclass):
    if isinstance(subclass, str):
        if subclass.endswith("14"):
            subclass = subclass[:-2]

        if "HPS" in subclass:
            subclass = subclass.replace(" (or equivalent)", "(or equivalent)")

        if subclass == "VERY LARGE AND EXTRA LARGE 115 KV/69 K":
            subclass = "VERY LARGE AND EXTRA LARGE 115 KV/69 KV"

    return subclass

df[("Customer Subclass", "")] = df[("Customer Subclass", "")].apply(clean_subclass)

In [81]:
def map_customer_class(group):
    class_mapping = {
        "Residential": ["0 TO 20 KWH","21 TO 50 KWH","51 TO 70 KWH",
                        "71 TO 100 KWH","101 TO 200 KWH","201 TO 300 KWH",
                        "301 TO 400 KWH","OVER 400 KWH"],
        "General Service A": ["0 TO 200 KWH","201 TO 300 KWH","301 TO 400 KWH", "OVER 400 KWH"],
        "General Service B": ["General Service B"],
        "General Power (GP) Secondary": ["MEDIUM SECONDARY", "LARGE SECONDARY", "VERY LARGE SECONDARY"],
        "GP 13.8 KV and below": ["MEDIUM 13.8 KV AND BELOW", "LARGE 13.8 KV AND BELOW", "VERY LARGE 13.8 KV AND BELOW"],
        "GP 34.5 KV": ["MEDIUM 34.5 KV", "LARGE 34.5 KV", "VERY LARGE AND EXTRA LARGE 34.5 KV"],
        "GP 115 KV / 69 KV": ["LARGE 115 KV/69 KV", "VERY LARGE AND EXTRA LARGE 115 KV/69 KV"],
        "GP 115 KV / 69 KV": ["GP 115 KV / 69 KV","LARGE 115 KV/69 KV","VERY LARGE AND EXTRA LARGE 115 KV/69 KV"],
        "GHMSCI": ["GHMSCI"]
    }

    customer_class = []
    lower_limit_demand = []
    upper_limit_demand = []
    lower_limit_consumption = []
    upper_limit_consumption = []
    residential_flag = True
    general_service_a_flag = False
    flat_streetlights_flag = False

    for subclass in group[("Customer Subclass", "")]:
        assigned_class = None
        lower_limit_d = None
        upper_limit_d = None
        consumption_lower_limit = None
        consumption_upper_limit = None

        if flat_streetlights_flag:
            assigned_class = "FLAT STREETLIGHTS"
        elif subclass == "Per kWh" or subclass == "125 W Mercury, 70 W HPS(or equivalent)":
            assigned_class = "FLAT STREETLIGHTS"
            flat_streetlights_flag = True
        elif residential_flag:
            assigned_class = "Residential"
            if subclass == "OVER 400 KWH":
                residential_flag = False
                general_service_a_flag = True
        elif general_service_a_flag:
            assigned_class = "General Service A"
            if subclass == "OVER 400 KWH":
                general_service_a_flag = False
        else:
            for class_name, subclasses in class_mapping.items():
                if subclass in subclasses:
                    assigned_class = class_name
                    break

        # Set the lower and upper limit demand based on the subclass
        if subclass == "General Service B":
            lower_limit_d = 5
            upper_limit_d = 40
        elif "MEDIUM" in subclass:
            lower_limit_d = 40
            upper_limit_d = 200
        elif "LARGE" in subclass:
            lower_limit_d = 200
            upper_limit_d = 750
        elif "VERY LARGE" in subclass:
            lower_limit_d = 750
            upper_limit_d = 10000
        elif "EXTRA LARGE" in subclass:
            lower_limit_d = 10000
            upper_limit_d = np.nan

        if "TO" in subclass:
            match = re.match(r'(\d+)\s+TO\s+(\d+)', subclass)
            if match:
                consumption_lower_limit = int(match.group(1))
                consumption_upper_limit = int(match.group(2))
        elif "OVER" in subclass:
            consumption_lower_limit = 400
            consumption_upper_limit = ""

        
        customer_class.append(assigned_class if assigned_class else "Unknown")
        lower_limit_demand.append(lower_limit_d)
        upper_limit_demand.append(upper_limit_d)
        lower_limit_consumption.append(consumption_lower_limit)
        upper_limit_consumption.append(consumption_upper_limit)

    group["Customer Class"] = customer_class
    group["Lower Limit Demand"] = lower_limit_demand
    group["Upper Limit Demand"] = upper_limit_demand
    group["Lower Limit Consumption"] = lower_limit_consumption
    group["Upper Limit Consumption"] = upper_limit_consumption
    return group

# Apply the mapping function group by group based on the Date
df = df.groupby(('Supply Period', ''), as_index=False).apply(map_customer_class)

# Reorder the columns
cols = [("Customer Class", ""), ("Customer Subclass", ""), 
        ("Lower Limit Demand", ""), ("Upper Limit Demand", ""), 
        ("Lower Limit Consumption", ""), ("Upper Limit Consumption", "")] + \
       [col for col in df.columns if col not in [("Customer Class", ""), ("Customer Subclass", ""), 
                                                 ("Lower Limit Demand", ""), ("Upper Limit Demand", ""), 
                                                 ("Lower Limit Consumption", ""), ("Upper Limit Consumption", "")]]

df = df[cols]

In [82]:
# Function to convert values as specified
def convert_value(val):
    if pd.isna(val) or val == "nan":
        return np.nan
    if isinstance(val, str):
        if val.startswith('(') and val.endswith(')'):
            return -float(val[1:-1])
        if val.endswith('%'):
            return float(val.strip('%')) / 100
        try:
            return float(val)
        except ValueError:
            return val  # Leave non-numeric strings as is
    return val

numeric_df = df.map(convert_value)
numeric_df = numeric_df.apply(pd.to_numeric, errors='coerce')

exclude_cols_kwh = ['Transmission Charge', 'Distribution Charge', 'Customer Class', 'Customer Subclass']
exclude_cols_kw = exclude_cols_kwh

combined_kwh = numeric_df.loc[:, numeric_df.columns.get_level_values(1) == 'per kWh'].drop(columns=exclude_cols_kwh, level=0).sum(axis=1, skipna=True)
combined_kw = numeric_df.loc[:, numeric_df.columns.get_level_values(1) == 'per kW'].drop(columns=exclude_cols_kw, level=0).sum(axis=1)

df[('kWh Attributable', '')] = combined_kwh
df[('kW Attributable', '')] = combined_kw

In [83]:
df.reset_index(drop=True, inplace=True)
df.head()

Charge Customer Class Customer Subclass Lower Limit Demand Upper Limit Demand  \
Unit                                                                            
0         Residential       0 TO 20 KWH                NaN                NaN   
1         Residential      21 TO 50 KWH                NaN                NaN   
2         Residential      51 TO 70 KWH                NaN                NaN   
3         Residential     71 TO 100 KWH                NaN                NaN   
4         Residential    101 TO 200 KWH                NaN                NaN   

Charge Lower Limit Consumption Upper Limit Consumption Generation Charge  \
Unit                                                             per kWh   
0                          0.0                      20            7.1291   
1                         21.0                      50            7.1291   
2                         51.0                      70            7.1291   
3                         71.0                     100            7.1291   
4                        101.0                     200            7.1291   

Charge Transmission Charge        System Loss Charge Distribution Charge  \
Unit               per kWh per kW            per kWh             per kWh   
0                   0.8650    NaN             0.6335              0.9803   
1                   0.8650    NaN             0.6335              0.9803   
2                   0.8650    NaN             0.6335              0.9803   
3                   0.8650    NaN             0.6335              0.9803   
4                   0.8650    NaN             0.6335              0.9803   

Charge        Supply Charge             Metering Charge              \
Unit   per kW       per kWh per cust/mo         per kWh per cust/mo   
0         NaN        0.4979       16.38          0.3350        5.00   
1         NaN        0.4979       16.38          0.3350        5.00   
2         NaN        0.4979       16.38          0.3350        5.00   
3         NaN        0.4979       16.38          0.3350        5.00   
4         NaN        0.4979       16.38          0.3350        5.00   

Charge Dist Rate True-Up 1 Dist Rate True-Up 2 Dist Rate True-Up 3  \
Unit               per kWh             per kWh             per kWh   
0                   0.0000            (0.1923)              0.0000   
1                   0.0000            (0.1923)              0.0000   
2                   0.0000            (0.1923)              0.0000   
3                   0.0000            (0.1923)              0.0000   
4                   0.0000            (0.1923)              0.0000   

Charge Dist Rate True-Up 4 Lifeline Rate Subsidy Senior Citizen Subsidy  \
Unit               per kWh               per kWh                per kWh   
0                 (0.8656)                   NaN                    NaN   
1                 (0.8656)                   NaN                    NaN   
2                 (0.8656)                   NaN                    NaN   
3                 (0.8656)                   NaN                    NaN   
4                 (0.8656)                0.0952                 0.0001   

Charge Current RPT Charge UC-ME for (NPC-SPUG) UC-ME for (RED-CI)   UC-EC  \
Unit              per kWh              per kWh            per kWh per kWh   
0                  0.0051               0.1783             0.0017  0.0000   
1                  0.0051               0.1783             0.0017  0.0000   
2                  0.0051               0.1783             0.0017  0.0000   
3                  0.0051               0.1783             0.0017  0.0000   
4                  0.0051               0.1783             0.0017  0.0000   

Charge   UC-SD Fit-All (Renewable) Lifeline Discount Special Discount  \
Unit   per kWh             per kWh                 %                %   
0       0.0428              0.0000           100.00%              NaN   
1       0.0428              0.0000            50.00%              NaN   
2       0.0428     

In [85]:
filename = f"MERALCO_Schedule_of_Rates_{date_part}.xlsx"

file_path = os.path.join(folder_path, filename)

df.to_excel(file_path, merge_cells=True)

### Rearranging Columns

In [86]:
from collections import defaultdict

df_copy = df.copy()

columns = df_copy.columns.tolist()

# Group columns by the second-level column name
grouped_columns = defaultdict(list)
for col in columns:
    if isinstance(col, tuple) and len(col) > 1:
        grouped_columns[col[1]].append(col)
    else:
        grouped_columns[col].append(col)

ordered_columns = []

for group_name, group in grouped_columns.items():
    # Add the group columns to the ordered list
    ordered_columns.extend(group)
    
    # If there's more than one column in the group and the group name is not empty, add a total column
    if group_name != "" and len(group) > 1:
        total_column_name = ('Total ' + group_name, '')
        
        if group_name == '%':
            df_copy[total_column_name] = df_copy[group].map(convert_value).sum(axis=1).mul(100)
        else:
            # Convert all values using convert_value function
            df_converted = df_copy[group].map(convert_value)
            # Ensure all values are numeric
            df_numeric = df_converted.apply(pd.to_numeric, errors='coerce')
            # Sum the values across the specified columns
            df_copy[total_column_name] = df_numeric.sum(axis=1)
        
        ordered_columns.append(total_column_name)

# Reorder the columns
df_copy = df_copy[ordered_columns]

df_copy.head()

Charge Customer Class Customer Subclass Lower Limit Demand Upper Limit Demand  \
Unit                                                                            
0         Residential       0 TO 20 KWH                NaN                NaN   
1         Residential      21 TO 50 KWH                NaN                NaN   
2         Residential      51 TO 70 KWH                NaN                NaN   
3         Residential     71 TO 100 KWH                NaN                NaN   
4         Residential    101 TO 200 KWH                NaN                NaN   

Charge Lower Limit Consumption Upper Limit Consumption Supply Period Start  \
Unit                                                                         
0                          0.0                      20            12-26-22   
1                         21.0                      50            12-26-22   
2                         51.0                      70            12-26-22   
3                         71.0                     100            12-26-22   
4                        101.0                     200            12-26-22   

Charge Supply Period End Supply Period   Date kWh Attributable  \
Unit                                                             
0               01-25-23         01-23  01-23           7.7655   
1               01-25-23         01-23  01-23           7.7655   
2               01-25-23         01-23  01-23           7.7655   
3               01-25-23         01-23  01-23           7.7655   
4               01-25-23         01-23  01-23           7.8608   

Charge kW Attributable Generation Charge Transmission Charge  \
Unit                             per kWh             per kWh   
0                  0.0            7.1291              0.8650   
1                  0.0            7.1291              0.8650   
2                  0.0            7.1291              0.8650   
3                  0.0            7.1291              0.8650   
4                  0.0            7.1291              0.8650   

Charge System Loss Charge Distribution Charge Supply Charge Metering Charge  \
Unit              per kWh             per kWh       per kWh         per kWh   
0                  0.6335              0.9803        0.4979          0.3350   
1                  0.6335              0.9803        0.4979          0.3350   
2                  0.6335              0.9803        0.4979          0.3350   
3                  0.6335              0.9803        0.4979          0.3350   
4                  0.6335              0.9803        0.4979          0.3350   

Charge Dist Rate True-Up 1 Dist Rate True-Up 2 Dist Rate True-Up 3  \
Unit               per kWh             per kWh             per kWh   
0                   0.0000            (0.1923)              0.0000   
1                   0.0000            (0.1923)              0.0000   
2                   0.0000            (0.1923)              0.0000   
3                   0.0000            (0.1923)              0.0000   
4                   0.0000            (0.1923)              0.0000   

Charge Dist Rate True-Up 4 Lifeline Rate Subsidy Senior Citizen Subsidy  \
Unit               per kWh               per kWh                per kWh   
0                 (0.8656)                   NaN                    NaN   
1                 (0.8656)                   NaN                    NaN   
2                 (0.8656)                   NaN                    NaN   
3                 (0.8656)                   NaN                    NaN   
4                 (0.8656)                0.0952                 0.0001   

Charge Current RPT Charge UC-ME for (NPC-SPUG) UC-ME for (RED-CI)   UC-EC  \
Unit              per kWh              per kWh            per kWh per kWh   
0                  0.0051               0.1783             0.0017  0.0000   
1                  0.0051               0.1783             0.0017  0.0000   
2                  0.0051               0.1783             0.0017  0.0000   
3                

In [89]:
filename = f"MERALCO_Schedule_of_Rates_{date_part} (reordered columns).xlsx"

file_path = os.path.join(folder_path, filename)

df_copy.to_excel(file_path, merge_cells=True)